<a href="https://colab.research.google.com/github/2020147544/Advances_in_Financial_Engineering/blob/main/chapter6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ensemble Methods

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import research as rs     # private module

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from scipy.special import comb

def cls_accuracy(N = 100, p = 1./3, k = 3.):
    #N, p, k = 100, 1./3, 3.
    p_ = 0

    for i in np.arange(0, int(N/k)+1):
        p_ += comb(N,i)*p**i*(1-p)**(N-i)
    if p > 1-p_ : 
        print("individual learners are considered poor classifier")
    else: 
        print("individual learners are not considered poor classifier")
    print(p, 1-p_)

cls_accuracy(N = 10, p = 1./3, k = 3.) # cls = 3, N estimate = 10
cls_accuracy(N = 100, p = 1./3, k = 3.) # cls = 3, N estimate = 100
cls_accuracy(N = 1001, p = 1./3, k = 3.) # cls = 3, N estimate = 1001

individual learners are not considered poor classifier
0.3333333333333333 0.4407356602143977
individual learners are not considered poor classifier
0.3333333333333333 0.4811966952738904
individual learners are not considered poor classifier
0.3333333333333333 0.5029710233411802


In [ ]:
#optional comparison with 2 class only

cls_accuracy(N = 100, p = 1./2, k = 2.) # cls = 2, N estimate = 100
cls_accuracy(N = 1001, p = 1./2, k = 2.) # cls = 2, N estimate = 1001

individual learners are considered poor classifier
0.5 0.46020538130641064
individual learners are not considered poor classifier
0.5 0.5000000000001502


## Exercise 6.3

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

#from sklearn.datasets import make_classification # create dataset
from sklearn.model_selection import train_test_split

dollar = pd.read_csv('/content/drive/MyDrive/Kp200F_volume_bars_500.csv', 
                 sep=',', 
                 header=0, 
                 parse_dates = True, 
                 index_col=['date_time'])

# For most part of the func we only use 'close'

close = dollar['close'].to_frame()
print(dollar)

ffd_series = close.apply(np.log).cumsum()
ffd_series = rs.fracDiff_FFD(ffd_series, # lack of research module
                    d = 1.99999889, 
                    thres=1e-5
                   ).dropna()

cs_event = rs.cs_filter(data = ffd_series, limit=(ffd_series.std() * 0.2))  # lack of research module

df_mtx = pd.DataFrame(index = cs_event).assign(close = close,
                                                ffd_series = ffd_series).drop_duplicates().dropna()
df_mtx

                                     tick_num    open    high     low   close  \
date_time                                                                       
2021-01-04 10:00:01.899669272+09:00        81  389.15  389.20  388.65  389.05   
2021-01-04 10:00:03.907859958+09:00       217  389.05  389.60  388.95  389.55   
2021-01-04 10:00:08.327160032+09:00       357  389.55  389.70  389.40  389.50   
2021-01-04 10:00:12.859526697+09:00       486  389.50  389.60  389.25  389.60   
2021-01-04 10:00:21.659149379+09:00       624  389.55  389.85  389.55  389.85   
...                                       ...     ...     ...     ...     ...   
2023-03-10 15:32:50.420869535+09:00  35364256  311.50  311.50  311.40  311.50   
2023-03-10 15:33:38.920764336+09:00  35364459  311.50  311.55  311.45  311.55   
2023-03-10 15:34:05.836568973+09:00  35364592  311.55  311.65  311.55  311.55   
2023-03-10 15:34:29.331336231+09:00  35364660  311.55  311.60  311.45  311.45   
2023-03-10 15:34:59.70091354

NameError: ignored

In [ ]:
df_mtx['volatility'] = rs.vol(df_mtx.close, span0 = 50) #one of our features, since we do not have a side

df_mtx.dropna(inplace = True)

vb = rs.vert_barrier(data = df_mtx.close, events = cs_event, period = 'days', freq = 5)

# triple barrier events based on filter while data is also based on filtered index
tb = rs.tri_barrier(data = df_mtx.close, 
                    events = cs_event, 
                    trgt = df_mtx['volatility'], 
                    min_req = 0.0002, 
                    num_threads = 3, 
                    ptSl= [2,2], #2x barriers
                    t1 = vb, 
                    side = None)

mlabel = rs.meta_label(data = df_mtx.close, 
                       events = tb, 
                       drop = 0.05) # because we do not have side, we need to drop rare labels

In [ ]:
mlabel['bin'].value_counts() #834

NameError: ignored

In [ ]:
X = df_mtx.reindex(mlabel.index)
Z = tb.reindex(mlabel.index)
y = mlabel['bin']

idx_Mat0 = rs.mp_idx_matrix(data = X.close, events = Z)

avgU = rs.av_unique(idx_Mat0).mean() #get ave uniqueness
print("Ave Uniqueness of Observations", avgU)

NameError: ignored

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=None)

# Benchmarks
rf_clf0 = RandomForestClassifier(n_estimators = 1000, 
                                bootstrap=True, 
                                n_jobs=1,
                                random_state=42,
                                oob_score=True)

base_estimate0 = DecisionTreeClassifier()


bag_clf0 = BaggingClassifier(base_estimator = base_estimate0,
                                n_estimators = 1000,
                                bootstrap=True, 
                                n_jobs=1, 
                                random_state=42,
                                oob_score=True)


rf_clf0.fit(X_train, y_train)
bag_clf0.fit(X_train, y_train)

print('Default rf Out-of-bag score: {}\n'.format(rf_clf0.oob_score_))
print('Default dt Out-of-bag score: {}\n'.format(bag_clf0.oob_score_))

NameError: ignored

In [ ]:
#based on book recommendation
rf_clf = RandomForestClassifier(n_estimators = 1000,
                                criterion = "entropy",
                                max_samples=avgU, #note averge unique used
                                bootstrap=True, 
                                n_jobs=1,
                                random_state=42,
                                class_weight="balanced_subsample",
                                oob_score=True)

clf = DecisionTreeClassifier(criterion = "entropy", 
                             max_features="auto", 
                             class_weight="balanced")

bag_clf = BaggingClassifier(base_estimator = clf,
                            n_estimators = 1000, 
                            max_samples=avgU, #note averge unique used
                            bootstrap=True, 
                            n_jobs=1,
                            random_state=42,
                            oob_score=True)

rf_clf.fit(X_train, y_train)
bag_clf.fit(X_train, y_train)

print('rf_clf rf Out-of-bag score: {}\n'.format(rf_clf.oob_score_))
print('bag_clf Out-of-bag score: {}\n'.format(bag_clf.oob_score_))

NameError: ignored

In [ ]:
#based on book recommendation
rf_clf1 = RandomForestClassifier(n_estimators = 1000,
                                criterion = "entropy",
                                bootstrap=True, 
                                n_jobs=1,
                                random_state=42,
                                class_weight="balanced_subsample",
                                oob_score=True)

clf1 = DecisionTreeClassifier(criterion = "entropy",
                             splitter="random", #added random as splitter, which was in rf but not in dt
                             max_features=None, 
                             class_weight="balanced")

bag_clf1 = BaggingClassifier(base_estimator = clf1,
                            n_estimators = 850, 
                            max_samples=avgU, #note averge unique used
                            bootstrap=True, 
                            n_jobs=1,
                            random_state=42,
                            oob_score=True)

rf_clf1.fit(X_train, y_train)
bag_clf1.fit(X_train, y_train)

print('rf_clf rf Out-of-bag score: {}\n'.format(rf_clf1.oob_score_))
print('bag_clf Out-of-bag score: {}\n'.format(bag_clf1.oob_score_))

NameError: ignored

In [ ]:
# if N is too small
cls_accuracy(N = 1, p = 1./3, k = 3.)
cls_accuracy(N = 1, p = 1./5, k = 5.)
cls_accuracy(N = 1, p = 1./10, k = 10.)

individual learners are considered poor classifier
0.3333333333333333 0.33333333333333326
individual learners are considered poor classifier
0.2 0.19999999999999996
individual learners are considered poor classifier
0.1 0.09999999999999998


In [ ]:
# if N is too large
cls_accuracy(N = 1200, p = 1./3, k = 3.)
cls_accuracy(N = 1200, p = 1./5, k = 5.)
cls_accuracy(N = 1200, p = 1./10, k = 10.)

individual learners are considered poor classifier
0.3333333333333333 -inf
individual learners are not considered poor classifier
0.2 0.48273446631083894
individual learners are not considered poor classifier
0.1 0.475713305287882


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

accuracy_array = np.zeros(5)

skf = StratifiedKFold(n_splits=5, 
                      shuffle=True, #shuffle = True
                      random_state = 42)

i = 0
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    rf_clf0.fit(X_train, y_train.values.ravel()) #use the original rf cls
    y_pred_rf = rf_clf0.predict_proba(X_test)[:, 1] #True positive only
    y_pred = rf_clf0.predict(X_test)
    accuracy_array[i] = accuracy_score(y_test, y_pred)
    i += 1

print("Mean Strat KFold accuracy with shuffle: {:.8f}".format(np.mean(accuracy_array)))

NameError: ignored